In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy and host a Stable Diffusion model on Vertex AI


<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/stable_diffusion/dreambooth_stable_diffusion.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/stable_diffusion/dreambooth_stable_diffusion.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
 <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/stable_diffusion/dreambooth_stable_diffusion.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This notebook demonstrates how to deploy and host a fine-tuned [Stable Diffusion 1.5](https://huggingface.co/runwayml/stable-diffusion-v1-5) model on Vertex AI. For hosting, you use the PyTorch 3 container built for Vertex AI with [TorchServe](https://pytorch.org/serve/index.html).

### Objective

In this tutorial, you learn how to host and deploy a Stable Diffusion 1.5 model on Vertex AI.

This tutorial uses the following Google Cloud ML services:

+ Vertex AI `Model` resource
+ Vertex AI `Endpoint` resource

The steps performed include:

+ Create a `torchserve` handler for responding to prediction requests.
+ Upload a Stable Diffusion 1.5 model on a prebuilt PyTorch container in Vertex AI.
+ Deploy a custom model to a Vertex AI Endpoint.
+ Send requests to the endpoint and parse the responses using Vertex AI Prediction service.

### Dataset

This notebook uses a collection of model artifacts fine-tuned to generate images of a small dog. These are the same images used in the original [DreamBooth paper](https://dreambooth.github.io/).

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI models
* Vertex AI endpoints
* Vertex AI prediction
* Google Cloud Storage
* (Optionally) Vertex AI Workbench

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Hardware requirements

This notebook requires that you use a GPU with a sufficient amount of VRAM available. It was tested on a `Tesla P100-PCIE-16GB` with 16 MiB of VRAM. Run the following cell to ensure that you have the correct hardware configuration.

In [2]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format="csv,noheader"

Tesla P100-PCIE-16GB, 16384 MiB, 16280 MiB


### Create a user-managed notebook on Vertex AI

If you are using Vertex AI Workbench, you can create a notebook with the correct configuration by doing the following:

+ Go to [Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench/user-managed) in the Google Cloud Console.
+ Click **New Notebook** and then click **Python 3 (CUDA Toolkit 11.0)** > **With 1 NVIDIA T4**.
+ In the **New notebook** dialog box, click **Advanced Options**. The **Create a user-managed notebook** page opens up.
+ In the **Create a user-managed notebook** page, do the following:
  * In the **Notebook name** box, type a name for your notebook, for example "my-stablediffusion-nb".
  * In the **Machine type** drop-down, select **N1-standard** > **n1-standard-16**.
  * In the **GPU type** drop-down, select **NVIDIA Tesla P100**.
  * Check the box next to **Install NVIDIA GPU driver automatically for me**
  * Expand **Disk(s)** and do the following:
    - Under **Boot disk type**, select **SSD Persistent Disk**.
    - Under **Data disk type**, select **SSD Persistent Disk**.
  * Click **Create**.

<div style="background:#feefe3; padding:5px; color:#aa0000">
<strong>Caution:</strong> Using a Vertex AI Workbench notebook with the above configuration can increase your costs significantly. You can estimage your costs using the <a href="https://cloud.google.com/products/calculator"><u>costs calculator</u></a>.</div>

## Installation

Install the following packages required to execute this notebook.

**Note**: You might need to change the version of PyTorch (`torch`) installed by `pip`.

In [1]:
%%writefile requirements.txt
ftfy
google-cloud-aiplatform
gradio
natsort
ninja
tensorboard==1.15.0
torch
torchaudio
torchvision
torchserve
torch-model-archiver
torch-workflow-archiver
transformers

Writing requirements.txt


In [2]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [3]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "us-central1"

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Import libraries

In [5]:
import base64
import gc
import math
import os
from glob import glob

import torch
from google.cloud import aiplatform
from IPython.display import display
from natsort import natsorted
from PIL import Image
from torch import autocast

OSError: /opt/conda/lib/python3.7/site-packages/nvidia/cublas/lib/libcublas.so.11: symbol cublasLtHSHMatmulAlgoInit version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference

## View model inferences

Before uploading the model to Vertex AI, you can review the expected output from the model. The model used in this notebook is available for your use and can be downloaded from Google Cloud Storage.

In [8]:
MODEL_ARTIFACTS = "model_artifacts"

if not os.path.exists(MODEL_ARTIFACTS):
    os.mkdir(MODEL_ARTIFACTS)

!gsutil -m cp gs://cloud-samples-data/vertex-ai/model-deployment/models/stable-diffusion/model.zip \
   $MODEL_ARTIFACTS

Copying gs://cloud-samples-data/vertex-ai/model-deployment/models/stable-diffusion/model.zip...
/ [1/1 files][  2.0 GiB/  2.0 GiB] 100% Done  56.9 MiB/s ETA 00:00:00           
Operation completed over 1 objects/2.0 GiB.                                      


### Consolidate model artifacts

Once your Stable Diffusion model is done training / fine tuning, you can look at the results! In the next step, you'll need to move the most recent directory of weights for you model into directory with the rest of your model artifacts.

In [ ]:
# @markdown Specify the weights directory to use (leave blank for latest)
WEIGHTS_DIR = ""  # @param {type:"string"}
if WEIGHTS_DIR == "":
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]

    print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

In [ ]:
!mv {WEIGHTS_DIR}/* {OUTPUT_DIR}

Now that you have all of the model artifacts in the same directory, we can create a model checkpoint (.ckpt).

In [ ]:
# @markdown Run conversion.
model_name = "model-dog"  # @param {type: "string"}
ckpt_path = OUTPUT_DIR + f"/{model_name}.ckpt"

half_arg = ""
# @markdown  Whether to convert to fp16, takes half the space (2GB), might lose some quality.
fp16 = True  # @param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_sd.py --model_path $OUTPUT_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

### Create new images

With everying in place, you can now generate new images from the Stable Diffusion model. First you must load your model into a `StableDiffusionPipeline`.

In [ ]:
model_path = ""
if model_path == "":
    model_path = OUTPUT_DIR

pipe = StableDiffusionPipeline.from_pretrained(
    model_path, torch_dtype=torch.float16
).to("cuda")
g_cuda = None

In [ ]:
g_cuda = torch.Generator(device="cuda")
seed = 52362
g_cuda.manual_seed(seed)

With the model loaded into a `StableDiffusionPipeline`, you can now generate results (inferences) from the model. Each set of inference requires an input (called a [prompt](https://learnprompting.org/)) that specifies what the model should create.

You can also vary other inputs into the model, as shown in the following cell.

In [ ]:
prompt = "photo of examplePup dog with snow mountain background."

num_samples = 4
num_batches = 1
num_columns = 2
guidance_scale = 10
num_inference_steps = 50
height = 512
width = 512

In [ ]:
def image_grid(imgs, cols):
    total = len(imgs)
    rows = math.ceil(total / cols)

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


all_images = []
for _ in range(num_batches):
    with autocast("cuda"):
        images = pipe(
            [prompt] * num_samples,
            height=height,
            width=width,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
        ).images
        all_images.extend(images)


grid = image_grid(all_images, num_columns)
grid

### Re-training the model

If you aren't satisfied with the inference outputs from the model, you can retrain it. Before you can do that, you might need to release the GPU memory.

In [ ]:
# Run this cell _only_ if you want to release the GPU memory.
pipe = None
gc.collect()
torch.cuda.empty_cache()

## Deploy the model to Vertex AI

You can host your Stable Diffusion 2.0 model on a Vertex AI endpoint where you can get inferences from it online. Uploading your model is a three step process: 

1. Create a custom TorchServe handler.
1. Upload the model artifacts onto Google Cloud Storage.
2. Create a Vertex AI model with the model artifacts and a prebuilt PyTorch container image.
3. Deploy the Vertex AI model onto an endpoint.

Before we can do that, though, we need to collect all of the model artifacts into a single place.

In [ ]:
!mkdir model_artifacts

### Create the custom TorchServe handler

The model deployed to Vertex AI uses [TorchServe](https://pytorch.org/serve/) to handle requests and return responses from the model. You must create a custom TorchServe handler to include in with the model artifacts uploaded to Vertex AI.

The handler file should be included in the directory with the other model artifacts.

In [ ]:
%%writefile model_artifacts/handler.py

"""Customized handler for Stable Diffusion 2."""
import base64
import logging
from io import BytesIO

import torch
from diffusers import EulerDiscreteScheduler
from diffusers import StableDiffusionPipeline
from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)
model_id = 'stabilityai/stable-diffusion-2'


class ModelHandler(BaseHandler):

  def __init__(self):
    self.initialized = False
    self.map_location = None
    self.device = None
    self.use_gpu = True
    self.store_avg = True
    self.pipe = None

  def initialize(self, context):
    """Initializes the pipe."""
    properties = context.system_properties
    gpu_id = properties.get('gpu_id')

    self.map_location, self.device, self.use_gpu = \
      ('cuda', torch.device('cuda:' + str(gpu_id)),
       True) if torch.cuda.is_available() else \
        ('cpu', torch.device('cpu'), False)

    # Use the Euler scheduler here instead
    scheduler = EulerDiscreteScheduler.from_pretrained(model_id,
                                                       subfolder='scheduler')
    pipe = StableDiffusionPipeline.from_pretrained(model_id,
                                                   scheduler=scheduler,
                                                   torch_dtype=torch.float16)
    pipe = pipe.to('cuda')
    # Uncomment the following line to reduce the GPU memory usage.
    # pipe.enable_attention_slicing()
    self.pipe = pipe

    self.initialized = True

  def preprocess(self, requests):
    """Noting to do here."""
    logger.info('requests: %s', requests)
    return requests

  def inference(self, preprocessed_data, *args, **kwargs):
    """Run the inference."""
    images = []
    for pd in preprocessed_data:
      prompt = pd['prompt']
      images.extend(self.pipe(prompt).images)
    return images

  def postprocess(self, output_batch):
    """Converts the images to base64 string."""
    postprocessed_data = []
    for op in output_batch:
      fp = BytesIO()
      op.save(fp, format='JPEG')
      postprocessed_data.append(base64.b64encode(fp.getvalue()).decode('utf-8'))
      fp.close()
    return postprocessed_data


After creating the handler file, you must package the handler as a model archiver (MAR) file. The output file must be named 'model.mar'.

In [ ]:
!torch-model-archiver \
  -f \
  --model-name model \
  --version 1.0 \
  --handler model_artifacts/handler.py \
  --export-path model_artifacts

### Upload the model artifacts to Google Cloud Storage

Create a new folder in your Google Cloud Storage bucket to hold the model artifacts

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}/"
FULL_GCS_PATH = f"{BUCKET_URI}model_artifacts"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Next, upload the model archive file and your trained Stable Diffusion 2.0 model to the folder on Cloud Storage.

In [ ]:
!gsutil cp -r model_artifacts $BUCKET_URI
!gsutil cp -r $OUTPUT_DIR $FULL_GCS_PATH

### Create the Vertex AI model

Once you've uploaded the model artifacts into a Cloud Storage bucket, you can create a new Vertex AI model. This notebook uses the [Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/start/use-vertex-ai-python-sdk) to create the model.

In [ ]:
PYTORCH_PREDICTION_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/pytorch-gpu.1-12:latest"
)
APP_NAME = "my-stable-diffusion"
VERSION = 1
MODEL_DISPLAY_NAME = "stable_diffusion_1_4-unique"
MODEL_DESCRIPTION = "stable_diffusion_1_4 container"
ENDPOINT_DISPLAY_NAME = f"{APP_NAME}-endpoint"

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    description=MODEL_DESCRIPTION,
    serving_container_image_uri=PYTORCH_PREDICTION_IMAGE_URI,
    artifact_uri=FULL_GCS_PATH,
)

model.wait()

print(model.display_name)
print(model.resource_name)

### Deploy the model to an endpoint

To get online preductions from your Stable Diffusion 2.0 model, you must [deploy it to a Vertex AI endpoint](https://cloud.google.com/vertex-ai/docs/predictions/overview). You can again use the Vertex AI SDK to create the endpoint and deploy your model.

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)

In [ ]:
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=MODEL_DISPLAY_NAME,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_P100",
    accelerator_count=1,
    traffic_percentage=100,
    deploy_request_timeout=1200,
    sync=True,
)

If the previous cell times out before returning, your endpoint might still be successfully deployed ot an end point. Check the [Cloud Console](https://console.cloud.google.com/vertex-ai/endpoints) to verify the results.

You can also extend the time to wait for deployment by changing the `deploy_request_timeout` argument passed to `model.deploy()`.

## Get online predictions

Finally, with your Stable Diffusion 2.0 model deployed to a Vertex AI endpoint, you can now get online predictions from it. Using the Vertex AI SDK, you only need a few lines of code to get an inference.

In [ ]:
instances = [{"prompt": "An examplePup dog with a baseball jersey."}]
response = endpoint.predict(instances=instances)

with open("img5.jpg", "wb") as g:
    g.write(base64.b64decode(response.predictions[0]))

display.Image("img5.jpg")

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete endpoint resource
endpoint.undeploy_all()
endpoint.delete()

# Delete model resource

model.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI